In [17]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm

## Preliminary Analysis

In [3]:
images = pd.read_csv("../data/tiny-imagenet-200/tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [3]:
#takes a while to run
images_expanded = pd.DataFrame(columns = ['Id', 'label'])
for img in images['Id']: 
    classes = images[images['Id'] == img]['labels']
    
    for img_class in classes: 
        images_expanded = images_expanded.append({'Id': img, 'label' : img_class}, ignore_index = True)

KeyboardInterrupt: 

In [ ]:
images_expanded.to_csv('words_expande.csv')

In [4]:
data_dir = pathlib.Path('../data/tiny-imagenet-200/tiny-imagenet-200/')
image_count = len(list(data_dir.glob('**/*.JPEG')))

In [5]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200

## Testing Inception v3 out of the box

In [6]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [7]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True

In [7]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)

Using cache found in C:\Users\Varun Jadia/.cache\torch\hub\pytorch_vision_v0.9.0


In [8]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [9]:
#inception expects input size 3*299*299
input_size = 299

In [10]:
from PIL import Image

In [11]:
#test on one image
input_image_one = Image.open("../data/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#converts image to 3*299*299
input_tensor_one = preprocess(input_image_one)

#convert to batches
input_batch = torch.cat((input_tensor_one.unsqueeze(0), input_tensor_one.unsqueeze(0)))

with torch.no_grad():
    output = model(input_batch)

In [12]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([[0.5844, 0.5588, 0.3677, 0.5314, 0.5706, 0.6019, 0.4203, 0.4587, 0.4384,
         0.4523, 0.5917, 0.3619, 0.5725, 0.5263, 0.5770, 0.4273, 0.5186, 0.4732,
         0.4843, 0.4914, 0.4898, 0.5532, 0.3608, 0.4117, 0.5177, 0.4508, 0.4183,
         0.4943, 0.3829, 0.5474, 0.4819, 0.4926, 0.3652, 0.5660, 0.5301, 0.4787,
         0.5190, 0.5015, 0.4638, 0.4591, 0.5849, 0.5683, 0.5857, 0.4282, 0.4621,
         0.4564, 0.4820, 0.4796, 0.6487, 0.4802, 0.4647, 0.4301, 0.5059, 0.5488,
         0.4507, 0.4437, 0.5217, 0.5219, 0.5793, 0.4818, 0.4189, 0.5636, 0.5038,
         0.4650, 0.5408, 0.5289, 0.4281, 0.5002, 0.4047, 0.4398, 0.4407, 0.4156,
         0.5662, 0.3598, 0.5238, 0.4666, 0.5090, 0.5307, 0.6265, 0.4710, 0.5150,
         0.4765, 0.4898, 0.5103, 0.4711, 0.5628, 0.5575, 0.4731, 0.4463, 0.4282,
         0.4772, 0.5152, 0.4514, 0.3888, 0.4413, 0.5423, 0.3222, 0.4871, 0.4790,
         0.5980, 0.5976, 0.5118, 0.5747, 0.6341, 0.5709, 0.3892, 0.4369, 0.4421,
         0.5213, 0.5090, 0.6

In [15]:
torch.topk(probabilities[0], 5)

torch.return_types.topk(
values=tensor([0.6487, 0.6341, 0.6265, 0.6243, 0.6242]),
indices=tensor([ 48, 103,  78, 182, 110]))

In [16]:
labels = pd.read_csv("../data/tiny-imagenet-200/wnids.txt", names = ['class_labels'], sep = '\t')
labels['predictions'] = probabilities[0]

In [17]:
labels.sort_values(by = ['predictions'], ascending = False)

,class_labels,predictions
48,n02233338,0.648720
103,n03126707,0.634147
78,n02099601,0.626471
182,n02085620,0.624325
110,n03014705,0.624234
...,...,...
11,n02106662,0.361941
22,n01443537,0.360817
73,n02423022,0.359763
114,n03670208,0.343643


### Updating feature extraction to match our dataset

In [8]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=1, is_inception=True):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training (no val for now)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [9]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [22]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    accuracies = []
    
    for phase in tqdm.tqdm(['train', 'val']): 
        for inputs, labels in tqdm.tqdm(dataloaders[phase]): 
            outputs, aux_outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
        phase_acc = running_corrects.double() / len(dataloaders[phase].dataset)
        print('Acc: {:.4f}'.format(phase, phase_acc))
        accuracies.append(phase_acc)

    return accuracies
            

In [11]:
##inception model to finetune
model_ft = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
# Handle the auxilary net, requires_grads automatically set to true 
num_ftrs = model_ft.AuxLogits.fc.in_features
model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net, requires_grad automatically set to true 
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
input_size = 299

In [12]:
##out of the box model
scratch_model = models.inception_v3(pretrained=True)
scratch_model.fc = nn.Linear(scratch_model.fc.in_features, 200)
scratch_model.AuxLogits.fc = nn.Linear(scratch_model.AuxLogits.fc.in_features, num_classes)

#### Check accuracy of out of the box model on all labeled data

In [15]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

In [23]:
out_box_acc = predict(dataloaders_dict, scratch_model)

  0%|                                                                                            | 0/2 [01:10<?, ?it/s]


KeyboardInterrupt: 

#### Retrain inception on transformed data and check accuracy on train/val data

In [21]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train']}

In [22]:
params_to_update = []
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [ ]:
##finetuned model accuracy
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)

Epoch 0/2
----------
